#  Multiple Inheritance


##  Table of Contents 
* [Multiple Inheritance using `super()`](#multiple-inheritance-using-super)
* [Method Resolution Order](#method_resolution_order)
* [Multiple Inheritance Alternatives](#multiple_inheritance_alternatives)
* [Conclusion](#conclusion)

---

Now that you’ve worked through an overview and some examples of `super()` and single inheritance, you will be introduced to an overview and some examples that will demonstrate how multiple inheritance works and how `super()` enables that functionality. 

<a class="anchor" id="multiple-inheritance-using-super"></a>
## Multiple Inheritance using `super()`

There is another use case in which `super()` really shines, and this one isn’t as common as the single inheritance scenario. In addition to single inheritance, Python supports multiple inheritance, in which a subclass can inherit from multiple superclasses that don’t necessarily inherit from each other (also known as **sibling classes**). 

The image below shows a very simple multiple inheritance scenario, where one class inherits from two unrelated (sibling) superclasses:

To better illustrate multiple inheritance in action, here is some code for you to try out, showing how you can build a right pyramid (a pyramid with a square base) out of a `Triangle` and a `Square`:

In [1]:
# Here we declare that the Square class inherits from the Rectangle class
class Square:
    def __init__(self, length):
        self.length = length

    def area(self):
        return self.length ** 2

    def perimeter(self):
        return 4 * self.length

In [2]:
class Triangle:
    def __init__(self, base, height):
        self.base = base
        self.height = height

    def area(self):
        return 0.5 * self.base * self.height

In [3]:
class RightPyramid(Triangle, Square):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

This example declares a `Triangle` class and a `RightPyramid` class that inherits from both `Square` and `Triangle`. 

You’ll see another `.area()` method that uses `super()` just like in single inheritance, with the aim of it reaching the `.perimeter()` and `.area()` methods defined all the way up in the `Rectangle` class. 

> **Note:** You may notice that the code above isn’t using any inherited properties from the `Triangle` class yet. Later examples will fully take advantage of inheritance from both `Triangle` and `Square`.

The problem, though, is that both superclasses (`Triangle` and `Square`) define a `.area()`. Take a second and think about what might happen when you call `.area()` on `RightPyramid`, and then try calling it like below:

In [4]:
pyramid = RightPyramid(2, 4)
pyramid.area()

AttributeError: 'RightPyramid' object has no attribute 'height'

Did you guess that Python will try to call `Triangle.area()`? This is because of something called the **method resolution order**.

<a class="anchor" id="method_resolution_order"></a>

## Method Resolution Order

The method resolution order (or **MRO**) tells Python how to search for inherited methods. This comes in handy when you’re using `super()` because the MRO tells you exactly where Python will look for a method you’re calling with `super()` and in what order.

Every class has an `.__mro__` attribute that allows us to inspect the order, so let’s do that:

In [ ]:
RightPyramid.__mro__

This tells us that methods will be searched first in `Rightpyramid`, then in `Triangle`, then in `Square`, then `Rectangle`, and then, if nothing is found, in `object`, from which all classes originate. 

The problem here is that the interpreter is searching for `.area()` in `Triangle` before `Square` and `Rectangle`, and upon finding `.area()` in `Triangle`, Python calls it instead of the one you want. Because `Triangle.area()` expects there to be a `.height` and a `.base` attribute, Python throws an `AttributeError`. 

Luckily, you have some control over how the MRO is constructed. Just by changing the signature of the `RightPyramid` class, you can search in the order you want, and the methods will resolve correctly:

In [ ]:
class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height
        super().__init__(self.base)

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

Notice that `RightPyramid` initializes partially with the `.__init__()` from the `Square` class. This allows `.area()` to use the `.length` on the object, as is designed. 

Now, you can build a pyramid, inspect the MRO, and calculate the surface area:

In [ ]:
pyramid = RightPyramid(2, 4)
RightPyramid.__mro__

In [ ]:
pyramid.area()

You see that the MRO is now what you’d expect, and you can inspect the area of the pyramid as well, thanks to `.area()` and `.perimeter()`. 

There’s still a problem here, though. For the sake of simplicity, I did a few things wrong in this example: the first, and arguably most importantly, was that I had two separate classes with the same method name and signature. 

This causes issues with method resolution, because the first instance of `.area()` that is encountered in the MRO list will be called. 

When you’re using `super()` with multiple inheritance, it’s imperative to design your classes to **cooperate**. Part of this is ensuring that your methods are unique so that they get resolved in the MRO, by making sure method signatures are unique—whether by using method names or method parameters.

In this case, to avoid a complete overhaul of your code, you can rename the `Triangle` class’s `.area()` method to `.tri_area()`. This way, the area methods can continue using class properties rather than taking external parameters:

In [ ]:
class Triangle:
    def __init__(self, base, height):
        self.base = base
        self.height = height
        super().__init__()

    def tri_area(self):
        return 0.5 * self.base * self.height

Let’s also go ahead and use this in the `RightPyramid` class:

In [ ]:
class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height):
        self.base = base
        self.slant_height = slant_height
        super().__init__(self.base)

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

    def area_2(self):
        base_area = super().area()
        triangle_area = super().tri_area()
        return triangle_area * 4 + base_area

The next issue here is that the code doesn’t have a delegated `Triangle` object like it does for a `Square` object, so calling `.area_2()` will give us an `AttributeError` since `.base` and `.height` don’t have any values. 

You need to do two things to fix this:

1. All methods that are called with `super()` need to have a call to their superclass’s version of that method. This means that you will need to add `super().__init__()` to the `.__init__()` methods of `Triangle` and `Square`.

2. Redesign all the `.__init__()` calls to take a keyword dictionary. See the complete code below.

In [ ]:
class Rectangle:
    def __init__(self, length, width, **kwargs):
        self.length = length
        self.width = width
        super().__init__(**kwargs)

    def area(self):
        return self.length * self.width

    def perimeter(self):
        return 2 * self.length + 2 * self.width

In [ ]:
# Here we declare that the Square class inherits from 
# the Rectangle class
class Square(Rectangle):
    def __init__(self, length, **kwargs):
        print('Square calling...')
        super().__init__(length=length, width=length, **kwargs)

In [ ]:
class Cube(Square):
    def surface_area(self):
        face_area = super().area()
        return face_area * 6

    def volume(self):
        face_area = super().area()
        return face_area * self.length

In [ ]:
class Triangle:
    def __init__(self, base, height, **kwargs):
        print('Triangle calling...')
        self.base = base
        self.height = height
        super().__init__(**kwargs)

    def tri_area(self):
        return 0.5 * self.base * self.height

In [ ]:
class RightPyramid(Square, Triangle):
    def __init__(self, base, slant_height, **kwargs):
        self.base = base
        self.slant_height = slant_height
        kwargs["height"] = slant_height
        kwargs["length"] = base
        super().__init__(base=base, **kwargs)

    def area(self):
        base_area = super().area()
        perimeter = super().perimeter()
        return 0.5 * perimeter * self.slant_height + base_area

    def area_2(self):
        base_area = super().area()
        triangle_area = super().tri_area()
        return triangle_area * 4 + base_area

There are a number of important differences in this code:

- `kwargs` is modified in some places (such as `RightPyramid.__init__()`: This will allow users of these objects to instantiate them only with the arguments that make sense for that particular object.

- **Setting up named arguments before `**kwargs`:** You can see this in `RightPyramid.__init__()`. This has the neat effect of popping that key right out of the `**kwargs` dictionary, so that by the time that it ends up at the end of the MRO in the `object` class, `**kwargs` is empty.

Now, when you use these updated classes, you have this:

In [ ]:
pyramid = RightPyramid(base=2, slant_height=4)

In [ ]:
pyramid.area()

In [ ]:
pyramid.area_2()

It works! You’ve used `super()` to successfully navigate a complicated class hierarchy while using both inheritance and composition to create new classes with minimal reimplementation.

<a class="anchor" id="multiple_inheritance_alternatives"></a>

## Multiple Inheritance Alternatives

As you can see, multiple inheritance can be useful but also lead to very complicated situations and code that is hard to read. It’s also rare to have objects that neatly inherit everything from more than multiple other objects. 

If you see yourself beginning to use multiple inheritance and a complicated class hierarchy, it’s worth asking yourself if you can achieve code that is cleaner and easier to understand by using **composition** instead of inheritance.

There’s another technique that can help you get around the complexity of multiple inheritance while still providing many of the benefits. This technique is in the form of a specialized, simple class called a **mixin**. 

A mixin works as a kind of inheritance, but instead of defining an is-a relationship it may be more accurate to say that it defines an **includes-a** relationship. With a mix-in you can write a behavior that can be directly included in any number of other classes.

Below, you will see a short example using `VolumeMixin` to give specific functionality to our 3D objects—in this case, a volume calculation:

In [ ]:
class Rectangle:
    def __init__(self, length, width):
        self.length = length
        self.width = width

    def area(self):
        return self.length * self.width

class Square(Rectangle):
    def __init__(self, length):
        super().__init__(length, length)

class VolumeMixin:
    def volume(self):
        return self.area() * self.height

class Cube(VolumeMixin, Square):
    def __init__(self, length):
        super().__init__(length)
        self.height = length

    def face_area(self):
        return super().area()

    def surface_area(self):
        return super().area() * 6

In this example, the code was reworked to include a mixin called `VolumeMixin`. The mixin is then used by `Cube` and gives `Cube` the ability to calculate its volume, which is shown below:

In [ ]:
cube = Cube(2)

In [ ]:
cube.surface_area()

In [ ]:
cube.volume()

This mixin can be used the same way in any other class that has an area defined for it and for which the formula `area * height` returns the correct volume.

<a class="anchor" id="conclusion"></a>
## Conclusion

In this section, you learned how multiple inheritance works in Python, and techniques to combine `super()` with multiple inheritance. You also learned about how Python resolves method calls using the method resolution order (MRO), as well as how to inspect and modify the MRO to ensure appropriate methods are called at appropriate times. 

For more information about object-oriented programming in Python and using `super()`, check out these resources:

- [Official `super()` documentation](https://docs.python.org/3/library/functions.html#super)
- [Python’s `super()` Considered Super by Raymond Hettinger](https://rhettinger.wordpress.com/2011/05/26/super-considered-super/)
- [Object-Oriented Programming in Python 3](https://realpython.com/python3-object-oriented-programming/)
